# Stream rank statistics algorithms

## Problem statement

* Rank statistics are statistics that select, consider, or compute over percentiles on ordinal data types.
* In systems programming rank systems come to the forefront when considering performance characteristics of large systems. For example, the 99-percentile of response times for some type of user query.
* Computing this value with exact accuracy requires as many values as there are requests. For systems of sufficiently large scale, this is infeasible.
* Instead we may use a variety of algorithms for estimating percentiles, within certain confidence intervals.
* These algorithms are stream algorithms because they operate on a contiuous stream of data, and rank algorithms because they aggregate via rank statistics. Thus, stream rank statistics algorithms.

# GK

* GK (Greenwald-Khanna) is a first example of such an algo.
* In GK we maintain an ordered list of tuples of the form $(v, g, \Delta)$. The ordering is in $v$.
* $v$ is a value from the data stream.
* $g$ is a value defined such that summing $g$ values from the bottom to a specific tuple gives you the minimum  rank for that tuple's $v$. The rank is numerical: e.g. it's an index position.
* $\Delta$ is likewise defined such that adding the sum of its values across the associated tuples to $g$ gives a maximum percentile estimate.
* $n$ is the number of values that have been seen thus far.
* The invariant conditions is that for any tuple, $g+\Delta \leq 2 \cdot \varepsilon \cdot n$. This a rank error boundary; we are stating that maximum (two-sided) error for the hypothesized rank be within the an error boundary conditioned on the chosen $\varepsilon$.
* Useful $\varepsilon$ values are dependent on the percentile sigfigs desired. 10 makes sense for a 90 percentile, and 99.9 requires a 0.01.
* Every new observation is inserted into the list in two steps.
* First, the `insert` routine pushes a new tuple into the list of the form $\{v, 1, \lfloor(2 * ε * n)\rfloor\}$ if the value is neither a minimum or maximum, and $\{v, 1, 0\}$ otherwise.
* Try running just the code block below to see just the `insert` operation in action:

In [102]:
from collections import namedtuple

Tuple = namedtuple('Tuple', 'v g Δ')

class GreenwaldKhanna:
    def __init__(self, ε):
        self.S = []
        self.ε = ε
        self.n = 0
        
    def insert(self, v):
        idx = 0
        for t in self.S:
            if (v < t.v):
                break
            else:
                idx += 1
        
        if (idx == 0 or idx == len(self.S)):
            Δ = 0
        else:
            Δ = round(2 * self.ε * self.n)
        
        tup = namedtuple('Tuple', 'v g Δ')
        self.S.insert(idx, tup(v=v, g=1, Δ=Δ))
        self.n += 1

In [107]:
GK = GreenwaldKhanna(0.1)
import numpy as np
np.random.seed(42)
for n in np.random.random(10):
    GK.insert(n)
GK.S

[Tuple(v=0.05808361216819946, g=1, Δ=0),
 Tuple(v=0.15599452033620265, g=1, Δ=0),
 Tuple(v=0.15601864044243652, g=1, Δ=0),
 Tuple(v=0.3745401188473625, g=1, Δ=0),
 Tuple(v=0.5986584841970366, g=1, Δ=1),
 Tuple(v=0.6011150117432088, g=1, Δ=2),
 Tuple(v=0.7080725777960455, g=1, Δ=2),
 Tuple(v=0.7319939418114051, g=1, Δ=0),
 Tuple(v=0.8661761457749352, g=1, Δ=1),
 Tuple(v=0.9507143064099162, g=1, Δ=0)]

* Because $\Delta$ is a function of the number of values inserted thus far, it will grow linearly with the number of values inserted, starting at 0 and going up to $\lfloor 2 \cdot \varepsilon \cdot n \rfloor$.
* The boundaries produced by this version of `insert` are technically correct, but useless, because the delta-bounds are uselessly large (try it out yourself by changing the number of random values generated in the code above).
* The true heart of this algorithm is the `compress` routine. This routine limits the number of tuples included in the dataset, by retaining only the essential information.
* Essentially it is just a method call at the end of `insert` that removes tuples that removes tuples from the list by smudging the boundaries of the tuples on either side of it, so long as the resulting error boundary does not exceed the boundary set by the invariant condition.

In [197]:
from collections import namedtuple

Tuple = namedtuple('Tuple', 'v g Δ')

class GreenwaldKhanna:
    def __init__(self, ε):
        self.S = []
        self.ε = ε
        self.n = 0
        
    def insert(self, v):
        idx = 0
        for t in self.S:
            if (v < t.v):
                break
            else:
                idx += 1
        
        if (idx == 0 or idx == len(self.S)):
            Δ = 0
        else:
            Δ = round(2 * self.ε * self.n)
        
        tup = namedtuple('Tuple', 'v g Δ')
        self.S.insert(idx, tup(v=v, g=1, Δ=Δ))
        self.n += 1
        self.compress()
        
    def compress(self):
        i = 0
        while(i < len(self.S) - 1):
            if(self.S[i].g + self.S[i + 1].g + self.S[i + 1].Δ <= round(2 * self.ε * self.n)):
                prior = self.S[i + 1]
                self.S[i + 1] = Tuple(v=prior.v, g=prior.g + self.S[i].g, Δ=self.S[i].Δ)
                del self.S[i]
                i -= 1
            i += 1

Witness this tuple volume reduction:

In [200]:
GK = GreenwaldKhanna(0.1)
import numpy as np
np.random.seed(42)
for n in np.random.random(1000):
    GK.insert(n)

In [201]:
GK.S

[Tuple(v=0.15599452033620265, g=175, Δ=0),
 Tuple(v=0.19068772066366657, g=10, Δ=192),
 Tuple(v=0.1959828624191452, g=37, Δ=157),
 Tuple(v=0.24317219099945409, g=9, Δ=195),
 Tuple(v=0.2464020332391068, g=1, Δ=198),
 Tuple(v=0.25179905890695276, g=12, Δ=192),
 Tuple(v=0.27074467314355377, g=6, Δ=195),
 Tuple(v=0.2721451372299627, g=1, Δ=198),
 Tuple(v=0.2820345725713065, g=41, Δ=187),
 Tuple(v=0.297121715623175, g=3, Δ=193),
 Tuple(v=0.3180034749718639, g=31, Δ=179),
 Tuple(v=0.35567271646494913, g=2, Δ=196),
 Tuple(v=0.3609738969400268, g=1, Δ=199),
 Tuple(v=0.370472102791382, g=24, Δ=181),
 Tuple(v=0.3745401188473625, g=40, Δ=174),
 Tuple(v=0.38929561404197655, g=2, Δ=191),
 Tuple(v=0.42521350446923345, g=8, Δ=191),
 Tuple(v=0.42899402737501835, g=32, Δ=169),
 Tuple(v=0.44600577295795574, g=1, Δ=200),
 Tuple(v=0.4524395161326934, g=6, Δ=197),
 Tuple(v=0.45606998421703593, g=27, Δ=172),
 Tuple(v=0.49636624720123634, g=3, Δ=194),
 Tuple(v=0.4984421989290573, g=15, Δ=184),
 Tuple(v=0.511

* To get a percentile rank we can descend down the list and stop at the first tuple within our desired error boundaries.

In [211]:
def rank(GK, r):
    rMin = 0
    rMax = 0
    for tup in GK.S:
        rMin += tup.g
        rMax = rMin + tup.Δ
        
        # print(r - rMin, GK.ε * GK.n, rMax - r, GK.ε * GK.n)
        
        if (r - rMin <= GK.ε * GK.n) and (rMax - r <= GK.ε * GK.n):
            return tup.v

In [212]:
rank(GK, 900)

0.8154614284548342

* This algorithm has the best space performance amongst stream rank statistic algorithms.
* However, when $\varepsilon$ is small, it can get computationally expensive, as it requires a list scan.